In [145]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [146]:
!pip install pyspark

In [147]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [148]:
from pyspark.sql import functions as F
from pyspark.sql import Window

raw_sales = spark.read.csv('./raw_sales.csv',header = True, inferSchema = True)
raw_sales.count()

29580

In [149]:
raw_sales.show()

+-------------------+--------+-------+------------+--------+
|           datesold|postcode|  price|propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|
|2007-06-27 00:00:00|    2906| 305000|       house|       3|
|2007-06-27 00:00:00|    2612| 850000|       house|       4|
|2007-06-28 00:00:00|   

In [150]:
wind_before = Window.partitionBy('postcode').orderBy('datesold').rowsBetween(-10, Window.currentRow - 1)
wind_after = Window.partitionBy('postcode').orderBy('datesold').rowsBetween(Window.currentRow + 1, 10)
wind_last = Window.orderBy('datesold').rowsBetween(Window.currentRow - 1, Window.currentRow - 1)

# Добавили 3 поля согласно первому заданию
raw_sales_mod = raw_sales.withColumn('last_house_price', F.sum('price').over(wind_last))
raw_sales_mod = raw_sales_mod.withColumn('avg_10_before', F.round(F.avg('price').over(wind_before), 2))
raw_sales_mod = raw_sales_mod.withColumn('avg_10_after', F.round(F.avg('price').over(wind_after), 2))

In [151]:
raw_sales_mod.show(100)

21/08/18 10:48:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------+-------+------------+--------+----------------+-------------+------------+
|           datesold|postcode|  price|propertyType|bedrooms|last_house_price|avg_10_before|avg_10_after|
+-------------------+--------+-------+------------+--------+----------------+-------------+------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|          900000|         null|    708350.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|          625000|     327000.0|    698350.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|          620000|     558500.0|    679350.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|          580000|    647333.33|    742850.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|          445000|     564250.0|    786600.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|          357000|     509900.0|    839200.0|
|2008-06-19 00:00:00|    2600| 765000|       house|    

## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [152]:
# Рассчитали среднегодовую стоимость жилья
avg_price_year = raw_sales.select(F.year('datesold').alias('yearsold'), 'datesold', 'price') \
                          .groupBy('yearsold').agg(F.round(F.avg('price'), 2).alias('avg_price_year')) \
                          .orderBy('yearsold') \
#                          .show()

# Получили итоговую таблицу после присоединения информации о средней стоимости в году, когда была совершена сделка
raw_sales_res = raw_sales_mod.join(avg_price_year, on=[F.year(raw_sales_mod['datesold']) == avg_price_year['yearsold']], how='left') \
                            .select('price', 'last_house_price', 'avg_10_before', 'avg_10_after', 'avg_price_year',
                                    'datesold', 'postcode', 'propertyType', 'bedrooms')
raw_sales_res.show()

21/08/18 10:48:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+
|  price|last_house_price|avg_10_before|avg_10_after|avg_price_year|           datesold|postcode|propertyType|bedrooms|
+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+
| 327000|          900000|         null|    708350.0|     522377.21|2007-07-08 00:00:00|    2600|       house|       1|
| 790000|          625000|     327000.0|    698350.0|     522377.21|2007-08-16 00:00:00|    2600|       house|       4|
| 825000|          620000|     558500.0|    679350.0|     522377.21|2007-12-05 00:00:00|    2600|       house|       3|
| 315000|          580000|    647333.33|    742850.0|     493814.16|2008-01-21 00:00:00|    2600|        unit|       1|
| 292500|          445000|     564250.0|    786600.0|     493814.16|2008-04-24 00:00:00|    2600|       house|       1|
| 329000|          357000|     509900.0|

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [153]:
raw_sales_res.show()

21/08/18 10:49:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+
|  price|last_house_price|avg_10_before|avg_10_after|avg_price_year|           datesold|postcode|propertyType|bedrooms|
+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+
| 327000|          900000|         null|    708350.0|     522377.21|2007-07-08 00:00:00|    2600|       house|       1|
| 790000|          625000|     327000.0|    698350.0|     522377.21|2007-08-16 00:00:00|    2600|       house|       4|
| 825000|          620000|     558500.0|    679350.0|     522377.21|2007-12-05 00:00:00|    2600|       house|       3|
| 315000|          580000|    647333.33|    742850.0|     493814.16|2008-01-21 00:00:00|    2600|        unit|       1|
| 292500|          445000|     564250.0|    786600.0|     493814.16|2008-04-24 00:00:00|    2600|       house|       1|
| 329000|          357000|     509900.0|

In [154]:
@F.udf('string')
def unique(value):
    value = list(map(str, value))
    return len(set(value))

# Модифицировали итоговый фрейм, добавив информацию о количестве уникальных значениях в каждой строчке.
# raw_sales_res.select('*', unique(F.array(raw_sales_res.columns)).alias('Unique_values')).show()
raw_sales_res = raw_sales_res.withColumn('Unique_values', unique(F.array(raw_sales_res.columns)))
raw_sales_res.show()

21/08/18 10:49:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+-------------+
|  price|last_house_price|avg_10_before|avg_10_after|avg_price_year|           datesold|postcode|propertyType|bedrooms|Unique_values|
+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+-------------+
| 327000|          900000|         null|    708350.0|     522377.21|2007-07-08 00:00:00|    2600|       house|       1|            9|
| 790000|          625000|     327000.0|    698350.0|     522377.21|2007-08-16 00:00:00|    2600|       house|       4|            9|
| 825000|          620000|     558500.0|    679350.0|     522377.21|2007-12-05 00:00:00|    2600|       house|       3|            9|
| 315000|          580000|    647333.33|    742850.0|     493814.16|2008-01-21 00:00:00|    2600|        unit|       1|            9|
| 292500|          445000|     564250.0|    786600.0|     4938

In [155]:
raw_sales_res.filter(F.col('Unique_values') != 9).show()

21/08/18 10:49:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+-------------+
|  price|last_house_price|avg_10_before|avg_10_after|avg_price_year|           datesold|postcode|propertyType|bedrooms|Unique_values|
+-------+----------------+-------------+------------+--------------+-------------------+--------+------------+--------+-------------+
| 760000|          760000|     690700.0|    976200.0|     496092.03|2009-07-01 00:00:00|    2600|       house|       3|            8|
| 865000|          865000|     781200.0|    967100.0|     592653.76|2014-03-24 00:00:00|    2600|       house|       4|            8|
| 985000|          675000|     967700.0|    967700.0|     592653.76|2014-08-08 00:00:00|    2600|       house|       4|            8|
|1100000|         1100000|     912700.0|    946650.0|     592653.76|2014-11-01 00:00:00|    2600|       house|       4|            8|
| 930000|          930000|    1076950.0|   1011050.0|     5926